# 1. Package import

In [8]:
from jbfg.pandas_utils import *
import pickle
import pandas as pd
import random
import sys
import datetime

In [9]:
def tstamp(txt1=None,txt2=None):
    tm = datetime.datetime.now().strftime('%I:%M:%S %p')
    if txt1 != None:sys.stdout.write(f"{txt1} ")
    sys.stdout.write(f"[{tm}] ")
    if txt2 != None:sys.stdout.write(f"{txt2} ")

# 2. Dataset 불러오기 및 고객특성 cols make
 - 과거 구매 이력 및 구매 상품 정보 => 고객특성 cols 생성
   - 오가닉상품 구매 경험 여부
   - 오가닉상품 구매 횟수
   - division_name별 구매 횟수
   - 고객별 평균 재구매일수
   - 고객별 unique 구매상품 갯수
   - 고객별 구매시 평균 상품 갯수
   
 - 현재 구매 내역 + 고객특성 cols => train_df0, test_df0 생성

In [10]:
schema = 'l2_kjbk_extnl'

In [11]:
# 상품정보
df_info    = get_df("SELECT * FROM products_info"  , schema)
# train 과거 구매이력
df_train_p = get_df("SELECT * FROM train_x_past"   , schema)
# train 현재 구매내역
df_train_c = get_df("SELECT * FROM train_x_current", schema)
# test 과거 구매이력
df_test_p  = get_df("SELECT * FROM test_x_past"    , schema)
# test 현재 구매내역
df_test_c  = get_df("SELECT * FROM test_x_current" , schema)

# train 현재 구매ID
df_train_y = get_df("SELECT * FROM train_y", schema)
# test 과거 구매ID
df_test_y  = get_df("SELECT product_id FROM test_x_past", schema)

In [4]:
display(df_info)
display(df_train_p)
display(df_train_c)
display(df_test_p)
display(df_test_c)

display(df_train_y)
display(df_test_y)

,product_id,name,category_id,division_id,category_name,division_name
0,21709,Sparkling Lemon Water,115,7,water seltzer sparkling water,beverages
1,20995,Organic Broccoli Florets,116,1,frozen produce,frozen
2,33000,Pure Irish Butter,36,16,butter,dairy eggs
3,7781,Organic Sticks Low Moisture Part Skim Mozzarel...,21,16,packaged cheese,dairy eggs
4,47144,Unsweetened Original Almond Breeze Almond Milk,91,16,soy lactosefree,dairy eggs
...,...,...,...,...,...,...
95,47209,Organic Hass Avocado,24,4,fresh fruits,produce
96,21903,Organic Baby Spinach,123,4,packaged vegetables fruits,produce
97,21137,Organic Strawberries,24,4,fresh fruits,produce
98,13176,Bag of Organic Bananas,24,4,fresh fruits,produce


,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id
0,717029,196613,15,1,9,3.0,5,0,41220
1,2325772,196613,35,6,11,2.0,7,1,41220
2,642668,196613,69,2,16,2.0,11,1,4210
3,1326515,196613,61,3,19,3.0,3,1,4210
4,1437845,196613,36,3,15,4.0,16,0,4210
...,...,...,...,...,...,...,...,...,...
1253967,1902783,163819,47,0,19,13.0,2,1,24852
1253968,1969590,163819,42,1,15,6.0,3,1,24852
1253969,2549215,163819,9,1,11,7.0,1,1,24852
1253970,3103211,163819,39,1,14,7.0,3,1,24852


,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered
0,806422,41116,47,5,18,3.0,4,1
1,2146033,127113,45,1,10,7.0,24,1
2,3055498,140168,45,5,10,26.0,11,1
3,3267527,60604,52,0,16,8.0,2,1
4,3125992,136554,46,4,9,5.0,2,0
...,...,...,...,...,...,...,...,...
24419,3184000,174046,100,5,7,2.0,12,1
24420,1754722,6710,100,5,11,3.0,21,1
24421,1076973,150840,100,0,13,0.0,18,1
24422,1247049,99707,100,1,13,3.0,6,1


,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id
0,117004,186630,37,4,11,3.0,12,1,21709
1,128658,186630,20,2,10,6.0,4,1,21709
2,165882,186630,56,3,15,5.0,10,1,21709
3,180165,186630,17,1,16,4.0,4,1,21709
4,394948,186630,46,5,11,7.0,7,1,21709
...,...,...,...,...,...,...,...,...,...
541917,2672845,119134,39,1,14,17.0,4,1,24852
541918,2731678,119134,38,5,14,3.0,4,1,24852
541919,2860406,119134,9,5,9,8.0,1,1,24852
541920,3252013,119134,4,5,10,7.0,2,1,24852


,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered
0,3384204,37987,38,5,16,13.0,9,1
1,2578612,169022,38,6,10,7.0,4,1
2,188887,200090,45,5,16,30.0,5,1
3,646772,149416,39,1,13,13.0,15,1
4,300214,113836,42,3,10,17.0,1,1
...,...,...,...,...,...,...,...,...
16121,2047884,57077,46,5,15,8.0,2,1
16122,567625,83592,43,4,12,0.0,1,1
16123,596442,157740,49,6,10,3.0,2,1
16124,671102,92043,43,4,10,10.0,1,1


,product_id
0,21709
1,21709
2,21709
3,21709
4,21709
...,...
24419,24852
24420,24852
24421,24852
24422,24852


,product_id
0,21709
1,21709
2,21709
3,21709
4,21709
...,...
541917,24852
541918,24852
541919,24852
541920,24852


In [12]:
m_train = pd.merge(df_train_p, df_info, how='left', on='product_id')
m_test  = pd.merge(df_test_p , df_info, how='left', on='product_id')

In [6]:
display(m_train)
display(m_test)

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,name,category_id,division_id,category_name,division_name
0,717029,196613,15,1,9,3.0,5,0,41220,Organic Romaine Lettuce,83,4,fresh vegetables,produce
1,2325772,196613,35,6,11,2.0,7,1,41220,Organic Romaine Lettuce,83,4,fresh vegetables,produce
2,642668,196613,69,2,16,2.0,11,1,4210,Whole Milk,84,16,milk,dairy eggs
3,1326515,196613,61,3,19,3.0,3,1,4210,Whole Milk,84,16,milk,dairy eggs
4,1437845,196613,36,3,15,4.0,16,0,4210,Whole Milk,84,16,milk,dairy eggs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253967,1902783,163819,47,0,19,13.0,2,1,24852,Banana,24,4,fresh fruits,produce
1253968,1969590,163819,42,1,15,6.0,3,1,24852,Banana,24,4,fresh fruits,produce
1253969,2549215,163819,9,1,11,7.0,1,1,24852,Banana,24,4,fresh fruits,produce
1253970,3103211,163819,39,1,14,7.0,3,1,24852,Banana,24,4,fresh fruits,produce


,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,name,category_id,division_id,category_name,division_name
0,117004,186630,37,4,11,3.0,12,1,21709,Sparkling Lemon Water,115,7,water seltzer sparkling water,beverages
1,128658,186630,20,2,10,6.0,4,1,21709,Sparkling Lemon Water,115,7,water seltzer sparkling water,beverages
2,165882,186630,56,3,15,5.0,10,1,21709,Sparkling Lemon Water,115,7,water seltzer sparkling water,beverages
3,180165,186630,17,1,16,4.0,4,1,21709,Sparkling Lemon Water,115,7,water seltzer sparkling water,beverages
4,394948,186630,46,5,11,7.0,7,1,21709,Sparkling Lemon Water,115,7,water seltzer sparkling water,beverages
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541917,2672845,119134,39,1,14,17.0,4,1,24852,Banana,24,4,fresh fruits,produce
541918,2731678,119134,38,5,14,3.0,4,1,24852,Banana,24,4,fresh fruits,produce
541919,2860406,119134,9,5,9,8.0,1,1,24852,Banana,24,4,fresh fruits,produce
541920,3252013,119134,4,5,10,7.0,2,1,24852,Banana,24,4,fresh fruits,produce


In [13]:
def make_cols1(df):
    # unique train_cst df
    unq_cst = pd.DataFrame(df['cstno'].drop_duplicates().reset_index(drop=True))
    
    # Organic 여부
    df['is_organic'] = df['name'].str.contains('Organic')
    df.loc[df["is_organic"]==True ,"is_organic"] = 1
    df.loc[df["is_organic"]==False,"is_organic"] = 0
    
    # cstno별 Organic 구매건수
    org_cnt = df.groupby(['cstno','is_organic'])['product_id'].count().reset_index()
    org_cnt = org_cnt.rename(columns={'product_id':'orgnic_cnt'})
    org_cnt = org_cnt[org_cnt["is_organic"]==1]
    org_cnt.drop(columns = ['is_organic'], inplace=True)
    
    # 고객별 Organic 구매횟수 및 구매경험
    m1 = pd.merge(unq_cst, org_cnt, how='left', on='cstno')
    m1.loc[m1["orgnic_cnt"]  > 0 ,"orgnic_yn"] = 1
    m1.loc[m1["orgnic_cnt"] == 0 ,"orgnic_yn"] = 0
    
    # 고객별 division_name별 구매횟수
    group_div = df.groupby(['cstno','division_name'])['product_id'].count().reset_index()
    group_div = group_div.rename(columns={'product_id':'CNT'})
    group_div = pd.pivot_table(group_div,index='cstno',columns='division_name',values='CNT').reset_index().fillna(0)
    m2 = pd.merge(m1, group_div, how='left', on='cstno')
    m2 = m2.fillna(0)
    
    # 고객별 평균 재구매일수
    df['order_interval'] = pd.to_numeric(df['order_interval'])
    avg_reorder = df.groupby('cstno')['order_interval'].mean().reset_index()
    avg_reorder = avg_reorder.rename(columns={'order_interval':'avg_interval'})
    m3 = pd.merge(m2, avg_reorder, how='left', on='cstno')
    m3 = m3.fillna(0)
    
    # 고객별 unq 구매상품 갯수
    unq_items = pd.DataFrame()
    unq_items['total_items']  = df.groupby('cstno').size().astype(np.int)
    unq_items['all_products'] = df.groupby('cstno')['product_id'].apply(set)
    unq_items['total_unq_items'] = (unq_items.all_products.map(len)).astype(np.int16)
    m4 = pd.merge(m3, unq_items, how='left', on='cstno')
    m4 = m4.fillna(0)
    
    # 고객별 구매시 평균 상품 갯수
    order_cnt = df.groupby(['cstno','order_id'])['product_id'].count().reset_index().groupby('cstno')['order_id'].count().reset_index()
    order_cnt = order_cnt.rename(columns={'order_id':'oid_cnt'})
    m5 = pd.merge(m4, order_cnt, how='left', on='cstno')
    m5 = m5.fillna(0)
    m5['avg_items'] = m5['total_items'] / m5['oid_cnt']
    m5.drop(columns=['total_items','all_products','oid_cnt'], inplace=True)
    
    return m5

In [8]:
%%time 
make_cols1(m_train)

CPU times: user 6.87 s, sys: 1.17 s, total: 8.04 s
Wall time: 8.08 s


,cstno,orgnic_cnt,orgnic_yn,bakery,beverages,canned goods,dairy eggs,deli,frozen,meat seafood,pantry,produce,avg_interval,total_unq_items,avg_items
0,196613,117.0,1.0,0.0,7.0,0.0,40.0,1.0,0.0,0.0,0.0,155.0,2.320000,42,2.900000
1,32775,88.0,1.0,0.0,0.0,2.0,53.0,17.0,0.0,0.0,0.0,134.0,6.584158,34,3.678571
2,32780,69.0,1.0,0.0,33.0,0.0,2.0,0.0,2.0,0.0,1.0,75.0,7.178571,24,2.260000
3,131089,155.0,1.0,0.0,8.0,5.0,8.0,0.0,16.0,0.0,0.0,133.0,7.017857,30,3.333333
4,163860,75.0,1.0,0.0,4.0,0.0,21.0,0.0,0.0,0.0,0.0,90.0,5.754386,14,2.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5854,131031,42.0,1.0,0.0,5.0,0.0,76.0,0.0,0.0,0.0,0.0,25.0,8.285714,6,2.409091
5855,32735,27.0,1.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,1.0,74.0,3.487500,19,1.620000
5856,163813,35.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95.0,7.170213,6,2.065217
5857,131050,48.0,1.0,1.0,0.0,0.0,13.0,0.0,0.0,6.0,2.0,107.0,7.943548,37,3.071429


In [9]:
%%time 
make_cols1(m_test)

CPU times: user 3.3 s, sys: 337 ms, total: 3.63 s
Wall time: 3.64 s


,cstno,orgnic_cnt,orgnic_yn,bakery,beverages,canned goods,dairy eggs,deli,frozen,meat seafood,pantry,produce,avg_interval,total_unq_items,avg_items
0,186630,18.0,1.0,0.0,67.0,0.0,3.0,0.0,0.0,0.0,0.0,25.0,6.052632,16,2.794118
1,153668,156.0,1.0,0.0,10.0,0.0,12.0,0.0,0.0,0.0,0.0,163.0,10.326087,39,5.285714
2,126077,48.0,1.0,0.0,22.0,0.0,32.0,0.0,0.0,0.0,0.0,88.0,10.856115,17,4.057143
3,16032,191.0,1.0,0.0,13.0,1.0,49.0,2.0,2.0,0.0,0.0,187.0,10.077551,50,7.257143
4,71952,52.0,1.0,0.0,11.0,0.0,10.0,9.0,0.0,0.0,0.0,67.0,10.064516,20,2.771429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4136,196984,39.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,5.394737,1,1.000000
4137,114096,34.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,3.794118,1,1.000000
4138,115929,32.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,5.000000,2,1.028571
4139,71975,38.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.0,8.027027,1,1.000000


In [14]:
cst_info_train = make_cols1(m_train)
cst_info_test  = make_cols1(m_test)

In [15]:
train_df0 = pd.merge(df_train_c, cst_info_train, how='left', on='cstno')
test_df0  = pd.merge(df_test_p , cst_info_test , how='left', on='cstno')
test_fnl0 = pd.merge(df_test_c , cst_info_test , how='left', on='cstno')

In [12]:
display(train_df0)
display(test_df0)
display(test_fnl0)

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,orgnic_cnt,orgnic_yn,...,canned goods,dairy eggs,deli,frozen,meat seafood,pantry,produce,avg_interval,total_unq_items,avg_items
0,806422,41116,47,5,18,3.0,4,1,21.0,1.0,...,1.0,4.0,24.0,1.0,0.0,0.0,66.0,9.008264,23,2.928571
1,2146033,127113,45,1,10,7.0,24,1,3.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,77.0,7.009091,14,2.666667
2,3055498,140168,45,5,10,26.0,11,1,98.0,1.0,...,0.0,14.0,0.0,0.0,0.0,2.0,133.0,7.574194,36,3.833333
3,3267527,60604,52,0,16,8.0,2,1,15.0,1.0,...,0.0,1.0,3.0,3.0,1.0,1.0,70.0,7.255102,23,2.348837
4,3125992,136554,46,4,9,5.0,2,0,104.0,1.0,...,0.0,20.0,0.0,0.0,0.0,0.0,161.0,7.966292,36,4.209302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24419,3184000,174046,100,5,7,2.0,12,1,380.0,1.0,...,0.0,149.0,36.0,0.0,0.0,0.0,501.0,3.081662,52,7.111111
24420,1754722,6710,100,5,11,3.0,21,1,329.0,1.0,...,5.0,120.0,0.0,6.0,0.0,2.0,290.0,3.668571,45,5.343434
24421,1076973,150840,100,0,13,0.0,18,1,393.0,1.0,...,6.0,10.0,42.0,2.0,0.0,45.0,690.0,0.787532,50,8.030303
24422,1247049,99707,100,1,13,3.0,6,1,15.0,1.0,...,0.0,21.0,0.0,0.0,0.0,0.0,219.0,1.575000,10,2.878788


,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,orgnic_cnt,...,canned goods,dairy eggs,deli,frozen,meat seafood,pantry,produce,avg_interval,total_unq_items,avg_items
0,117004,186630,37,4,11,3.0,12,1,21709,18.0,...,0.0,3.0,0.0,0.0,0.0,0.0,25.0,6.052632,16,2.794118
1,128658,186630,20,2,10,6.0,4,1,21709,18.0,...,0.0,3.0,0.0,0.0,0.0,0.0,25.0,6.052632,16,2.794118
2,165882,186630,56,3,15,5.0,10,1,21709,18.0,...,0.0,3.0,0.0,0.0,0.0,0.0,25.0,6.052632,16,2.794118
3,180165,186630,17,1,16,4.0,4,1,21709,18.0,...,0.0,3.0,0.0,0.0,0.0,0.0,25.0,6.052632,16,2.794118
4,394948,186630,46,5,11,7.0,7,1,21709,18.0,...,0.0,3.0,0.0,0.0,0.0,0.0,25.0,6.052632,16,2.794118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541917,2672845,119134,39,1,14,17.0,4,1,24852,32.0,...,0.0,7.0,2.0,0.0,0.0,0.0,71.0,6.976190,21,2.000000
541918,2731678,119134,38,5,14,3.0,4,1,24852,32.0,...,0.0,7.0,2.0,0.0,0.0,0.0,71.0,6.976190,21,2.000000
541919,2860406,119134,9,5,9,8.0,1,1,24852,32.0,...,0.0,7.0,2.0,0.0,0.0,0.0,71.0,6.976190,21,2.000000
541920,3252013,119134,4,5,10,7.0,2,1,24852,32.0,...,0.0,7.0,2.0,0.0,0.0,0.0,71.0,6.976190,21,2.000000


,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,orgnic_cnt,orgnic_yn,...,canned goods,dairy eggs,deli,frozen,meat seafood,pantry,produce,avg_interval,total_unq_items,avg_items
0,3384204,37987,38,5,16,13.0,9,1,69.0,1.0,...,0.0,33.0,1.0,0.0,0.0,1.0,60.0,9.951456,30,3.228571
1,2578612,169022,38,6,10,7.0,4,1,147.0,1.0,...,0.0,45.0,0.0,0.0,0.0,0.0,169.0,6.610294,29,7.611111
2,188887,200090,45,5,16,30.0,5,1,8.0,1.0,...,0.0,48.0,0.0,0.0,0.0,0.0,5.0,7.295775,7,2.027778
3,646772,149416,39,1,13,13.0,15,1,106.0,1.0,...,1.0,30.0,0.0,0.0,0.0,0.0,113.0,8.993631,27,4.472222
4,300214,113836,42,3,10,17.0,1,1,89.0,1.0,...,2.0,7.0,0.0,0.0,3.0,2.0,128.0,9.489796,42,4.083333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16121,2047884,57077,46,5,15,8.0,2,1,38.0,1.0,...,0.0,3.0,1.0,5.0,0.0,0.0,101.0,7.700000,21,3.025000
16122,567625,83592,43,4,12,0.0,1,1,5.0,1.0,...,0.0,3.0,0.0,0.0,0.0,0.0,54.0,7.500000,9,1.425000
16123,596442,157740,49,6,10,3.0,2,1,24.0,1.0,...,0.0,29.0,0.0,1.0,0.0,0.0,64.0,7.752381,20,2.609756
16124,671102,92043,43,4,10,10.0,1,1,97.0,1.0,...,0.0,14.0,0.0,7.0,3.0,0.0,177.0,8.782383,32,4.902439


# 3. New cols make(구매내역)
 - 평일, 주말 여부
 - 오전, 오후 여부
 - 아침, 점심, 저녁시간 여부
 - 주문순서 구간

In [16]:
def weekday_yn(week_day):
    if week_day in ('1','2','3','4','5') : # 월화수목금
        return 1
    else:
        return 0
    
def weekend_yn(week_day):
    if week_day in ('6','0') : # 토일
        return 1
    else:
        return 0

def morning_yn(order_hour):
    if order_hour in ('0','1','2','3','4','5','6','7','8','9','10','11') : # 오전
        return 1
    else:
        return 0

def afternoon_yn(order_hour):
    if order_hour in ('12','13','14','15','16','17','18','19','20','21','22','23') : # 오후
        return 1
    else:
        return 0
    
def breakfast_yn(order_hour):
    if order_hour in ('6','7','8','9') :    # 아침
        return 1
    else:
        return 0
    
def lunch_yn(order_hour):
    if order_hour in ('11','12','13','14') : # 점심
        return 1
    else:
        return 0    
    
def dinner_yn(order_hour):
    if order_hour in ('17','18','19','20') : # 저녁
        return 1
    else:
        return 0
    
def order_nums(add_order):
    if add_order == '1':
        return 1
    if add_order in ('2','3','4','5'):
        return 2
    if add_order in ('6','7','8','9','10'):
        return 3
    if add_order in ('11','12','13','14','15','16','17','18','19','20'):
        return 4
    if add_order in ('21','22','23','24','25','26','27','28','29','30'):
        return 5
    if add_order in ('31','32','33','34','35','36','37','38','39','40','41','42','43','44','45','46','47','48','49','50'):
        return 6
    if add_order in ('51','52','53','54','55','56','57','58','59','60','61','62','63','64','65','66','67','68','69','70','71','72','73','74','75','76','77','78','79','80'):
        return 7
    if add_order in ('81','82','83','84','85','86','87','88','89','90','91','92','93','94','95','96','97','98','99','100'):
        return 8
    else:
        return 0

In [17]:
def make_cols2(df):
    df['weekday_yn']   = df.apply(lambda x: weekday_yn(x.week_day)    , axis=1)
    df['weekend_yn']   = df.apply(lambda x: weekend_yn(x.week_day)    , axis=1)
    df['morning_yn']   = df.apply(lambda x: morning_yn(x.order_hour)  , axis=1)
    df['afternoon_yn'] = df.apply(lambda x: afternoon_yn(x.order_hour), axis=1)
    df['breakfast_yn'] = df.apply(lambda x: breakfast_yn(x.order_hour), axis=1)
    df['lunch_yn']     = df.apply(lambda x: lunch_yn(x.order_hour)    , axis=1)
    df['dinner_yn']    = df.apply(lambda x: dinner_yn(x.order_hour)   , axis=1)
    df['order_nums']   = df.apply(lambda x: order_nums(x.add_order)   , axis=1)
    
    return df

In [15]:
%%time
make_cols2(train_df0)

CPU times: user 1.9 s, sys: 31.6 ms, total: 1.93 s
Wall time: 1.94 s


,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,orgnic_cnt,orgnic_yn,...,total_unq_items,avg_items,weekday_yn,weekend_yn,morning_yn,afternoon_yn,breakfast_yn,lunch_yn,dinner_yn,order_nums
0,806422,41116,47,5,18,3.0,4,1,21.0,1.0,...,23,2.928571,1,0,0,1,0,0,1,2
1,2146033,127113,45,1,10,7.0,24,1,3.0,1.0,...,14,2.666667,1,0,1,0,0,0,0,5
2,3055498,140168,45,5,10,26.0,11,1,98.0,1.0,...,36,3.833333,1,0,1,0,0,0,0,4
3,3267527,60604,52,0,16,8.0,2,1,15.0,1.0,...,23,2.348837,0,1,0,1,0,0,0,2
4,3125992,136554,46,4,9,5.0,2,0,104.0,1.0,...,36,4.209302,1,0,1,0,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24419,3184000,174046,100,5,7,2.0,12,1,380.0,1.0,...,52,7.111111,1,0,1,0,1,0,0,4
24420,1754722,6710,100,5,11,3.0,21,1,329.0,1.0,...,45,5.343434,1,0,1,0,0,1,0,5
24421,1076973,150840,100,0,13,0.0,18,1,393.0,1.0,...,50,8.030303,0,1,0,1,0,1,0,4
24422,1247049,99707,100,1,13,3.0,6,1,15.0,1.0,...,10,2.878788,1,0,0,1,0,1,0,3


In [16]:
%%time
make_cols2(test_df0)

CPU times: user 42.3 s, sys: 4.3 s, total: 46.6 s
Wall time: 46.7 s


,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,orgnic_cnt,...,total_unq_items,avg_items,weekday_yn,weekend_yn,morning_yn,afternoon_yn,breakfast_yn,lunch_yn,dinner_yn,order_nums
0,117004,186630,37,4,11,3.0,12,1,21709,18.0,...,16,2.794118,1,0,1,0,0,1,0,4
1,128658,186630,20,2,10,6.0,4,1,21709,18.0,...,16,2.794118,1,0,1,0,0,0,0,2
2,165882,186630,56,3,15,5.0,10,1,21709,18.0,...,16,2.794118,1,0,0,1,0,0,0,3
3,180165,186630,17,1,16,4.0,4,1,21709,18.0,...,16,2.794118,1,0,0,1,0,0,0,2
4,394948,186630,46,5,11,7.0,7,1,21709,18.0,...,16,2.794118,1,0,1,0,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541917,2672845,119134,39,1,14,17.0,4,1,24852,32.0,...,21,2.000000,1,0,0,1,0,1,0,2
541918,2731678,119134,38,5,14,3.0,4,1,24852,32.0,...,21,2.000000,1,0,0,1,0,1,0,2
541919,2860406,119134,9,5,9,8.0,1,1,24852,32.0,...,21,2.000000,1,0,1,0,1,0,0,1
541920,3252013,119134,4,5,10,7.0,2,1,24852,32.0,...,21,2.000000,1,0,1,0,0,0,0,2


In [18]:
train_df1 = make_cols2(train_df0)
test_df1  = make_cols2(test_df0)
test_fnl1 = make_cols2(test_fnl0)

In [18]:
display(train_df1)
display(test_df1)
display(test_fnl1)

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,orgnic_cnt,orgnic_yn,...,total_unq_items,avg_items,weekday_yn,weekend_yn,morning_yn,afternoon_yn,breakfast_yn,lunch_yn,dinner_yn,order_nums
0,806422,41116,47,5,18,3.0,4,1,21.0,1.0,...,23,2.928571,1,0,0,1,0,0,1,2
1,2146033,127113,45,1,10,7.0,24,1,3.0,1.0,...,14,2.666667,1,0,1,0,0,0,0,5
2,3055498,140168,45,5,10,26.0,11,1,98.0,1.0,...,36,3.833333,1,0,1,0,0,0,0,4
3,3267527,60604,52,0,16,8.0,2,1,15.0,1.0,...,23,2.348837,0,1,0,1,0,0,0,2
4,3125992,136554,46,4,9,5.0,2,0,104.0,1.0,...,36,4.209302,1,0,1,0,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24419,3184000,174046,100,5,7,2.0,12,1,380.0,1.0,...,52,7.111111,1,0,1,0,1,0,0,4
24420,1754722,6710,100,5,11,3.0,21,1,329.0,1.0,...,45,5.343434,1,0,1,0,0,1,0,5
24421,1076973,150840,100,0,13,0.0,18,1,393.0,1.0,...,50,8.030303,0,1,0,1,0,1,0,4
24422,1247049,99707,100,1,13,3.0,6,1,15.0,1.0,...,10,2.878788,1,0,0,1,0,1,0,3


,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,orgnic_cnt,...,total_unq_items,avg_items,weekday_yn,weekend_yn,morning_yn,afternoon_yn,breakfast_yn,lunch_yn,dinner_yn,order_nums
0,117004,186630,37,4,11,3.0,12,1,21709,18.0,...,16,2.794118,1,0,1,0,0,1,0,4
1,128658,186630,20,2,10,6.0,4,1,21709,18.0,...,16,2.794118,1,0,1,0,0,0,0,2
2,165882,186630,56,3,15,5.0,10,1,21709,18.0,...,16,2.794118,1,0,0,1,0,0,0,3
3,180165,186630,17,1,16,4.0,4,1,21709,18.0,...,16,2.794118,1,0,0,1,0,0,0,2
4,394948,186630,46,5,11,7.0,7,1,21709,18.0,...,16,2.794118,1,0,1,0,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541917,2672845,119134,39,1,14,17.0,4,1,24852,32.0,...,21,2.000000,1,0,0,1,0,1,0,2
541918,2731678,119134,38,5,14,3.0,4,1,24852,32.0,...,21,2.000000,1,0,0,1,0,1,0,2
541919,2860406,119134,9,5,9,8.0,1,1,24852,32.0,...,21,2.000000,1,0,1,0,1,0,0,1
541920,3252013,119134,4,5,10,7.0,2,1,24852,32.0,...,21,2.000000,1,0,1,0,0,0,0,2


,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,orgnic_cnt,orgnic_yn,...,total_unq_items,avg_items,weekday_yn,weekend_yn,morning_yn,afternoon_yn,breakfast_yn,lunch_yn,dinner_yn,order_nums
0,3384204,37987,38,5,16,13.0,9,1,69.0,1.0,...,30,3.228571,1,0,0,1,0,0,0,3
1,2578612,169022,38,6,10,7.0,4,1,147.0,1.0,...,29,7.611111,0,1,1,0,0,0,0,2
2,188887,200090,45,5,16,30.0,5,1,8.0,1.0,...,7,2.027778,1,0,0,1,0,0,0,2
3,646772,149416,39,1,13,13.0,15,1,106.0,1.0,...,27,4.472222,1,0,0,1,0,1,0,4
4,300214,113836,42,3,10,17.0,1,1,89.0,1.0,...,42,4.083333,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16121,2047884,57077,46,5,15,8.0,2,1,38.0,1.0,...,21,3.025000,1,0,0,1,0,0,0,2
16122,567625,83592,43,4,12,0.0,1,1,5.0,1.0,...,9,1.425000,1,0,0,1,0,1,0,1
16123,596442,157740,49,6,10,3.0,2,1,24.0,1.0,...,20,2.609756,0,1,1,0,0,0,0,2
16124,671102,92043,43,4,10,10.0,1,1,97.0,1.0,...,32,4.902439,1,0,1,0,0,0,0,1


# 4. 데이터 전처리
 - 문자형 -> 숫자형으로 변경(order_interval: 직전구매후경과일수, add_order: 주문서내담긴상품의순번, order_number: 고객별주문서순번)
 - 범주형 데이터 onehotencoding(week_day:구매요일 , is_reordered: 재구매여부)
 - columns merge

In [19]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values='null', strategy='constant', fill_value='missing')
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)

In [20]:
def make_cols3(df):
    # 문자형 -> 숫자형으로 변환
    df['order_interval'] = pd.to_numeric(df['order_interval'])
    df['add_order']      = pd.to_numeric(df['add_order'])
    df['order_number']   = pd.to_numeric(df['order_number'])
    
    # 범주형/일반형 cols 구분 
    cate_cols   = set(['week_day','is_reordered'])
    col         = set(df.columns.to_list())
    normal_cols = list(col-cate_cols)
    cate_cols   = list(cate_cols)
    
    # 범주형 cols 공백처리 후, onehotencoding 처리
    df_pp1 = imputer.fit_transform(df[cate_cols])
    df_pp2 = encoder.fit_transform(df_pp1)
    df_pp3 = pd.DataFrame(df_pp2, columns=encoder.get_feature_names(cate_cols))
    
    # 일반형col + 범주형(전처리후) => 최종 df
    dfF = pd.merge(df, df_pp3, how='left', left_index=True, right_index=True)
    
    return dfF

In [21]:
%%time
make_cols3(train_df1)

CPU times: user 52.2 ms, sys: 4.03 ms, total: 56.2 ms
Wall time: 55.4 ms


,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,orgnic_cnt,orgnic_yn,...,order_nums,week_day_0,week_day_1,week_day_2,week_day_3,week_day_4,week_day_5,week_day_6,is_reordered_0,is_reordered_1
0,806422,41116,47,5,18,3.0,4,1,21.0,1.0,...,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,2146033,127113,45,1,10,7.0,24,1,3.0,1.0,...,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3055498,140168,45,5,10,26.0,11,1,98.0,1.0,...,4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,3267527,60604,52,0,16,8.0,2,1,15.0,1.0,...,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,3125992,136554,46,4,9,5.0,2,0,104.0,1.0,...,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24419,3184000,174046,100,5,7,2.0,12,1,380.0,1.0,...,4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
24420,1754722,6710,100,5,11,3.0,21,1,329.0,1.0,...,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
24421,1076973,150840,100,0,13,0.0,18,1,393.0,1.0,...,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
24422,1247049,99707,100,1,13,3.0,6,1,15.0,1.0,...,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [21]:
train_df2 = make_cols3(train_df1)
test_df2  = make_cols3(test_df1)
test_fnl2 = make_cols3(test_fnl1)

In [23]:
display(train_df2)
display(test_df2)
display(test_fnl2)

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,orgnic_cnt,orgnic_yn,...,order_nums,week_day_0,week_day_1,week_day_2,week_day_3,week_day_4,week_day_5,week_day_6,is_reordered_0,is_reordered_1
0,806422,41116,47,5,18,3.0,4,1,21.0,1.0,...,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,2146033,127113,45,1,10,7.0,24,1,3.0,1.0,...,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3055498,140168,45,5,10,26.0,11,1,98.0,1.0,...,4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,3267527,60604,52,0,16,8.0,2,1,15.0,1.0,...,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,3125992,136554,46,4,9,5.0,2,0,104.0,1.0,...,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24419,3184000,174046,100,5,7,2.0,12,1,380.0,1.0,...,4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
24420,1754722,6710,100,5,11,3.0,21,1,329.0,1.0,...,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
24421,1076973,150840,100,0,13,0.0,18,1,393.0,1.0,...,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
24422,1247049,99707,100,1,13,3.0,6,1,15.0,1.0,...,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,orgnic_cnt,...,order_nums,week_day_0,week_day_1,week_day_2,week_day_3,week_day_4,week_day_5,week_day_6,is_reordered_0,is_reordered_1
0,117004,186630,37,4,11,3.0,12,1,21709,18.0,...,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,128658,186630,20,2,10,6.0,4,1,21709,18.0,...,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,165882,186630,56,3,15,5.0,10,1,21709,18.0,...,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,180165,186630,17,1,16,4.0,4,1,21709,18.0,...,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,394948,186630,46,5,11,7.0,7,1,21709,18.0,...,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541917,2672845,119134,39,1,14,17.0,4,1,24852,32.0,...,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
541918,2731678,119134,38,5,14,3.0,4,1,24852,32.0,...,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
541919,2860406,119134,9,5,9,8.0,1,1,24852,32.0,...,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
541920,3252013,119134,4,5,10,7.0,2,1,24852,32.0,...,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,orgnic_cnt,orgnic_yn,...,order_nums,week_day_0,week_day_1,week_day_2,week_day_3,week_day_4,week_day_5,week_day_6,is_reordered_0,is_reordered_1
0,3384204,37987,38,5,16,13.0,9,1,69.0,1.0,...,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,2578612,169022,38,6,10,7.0,4,1,147.0,1.0,...,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,188887,200090,45,5,16,30.0,5,1,8.0,1.0,...,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,646772,149416,39,1,13,13.0,15,1,106.0,1.0,...,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,300214,113836,42,3,10,17.0,1,1,89.0,1.0,...,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16121,2047884,57077,46,5,15,8.0,2,1,38.0,1.0,...,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
16122,567625,83592,43,4,12,0.0,1,1,5.0,1.0,...,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
16123,596442,157740,49,6,10,3.0,2,1,24.0,1.0,...,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
16124,671102,92043,43,4,10,10.0,1,1,97.0,1.0,...,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


# 5. Dataset 생성
 - Train set / Test set
 - Y_Train random sampling

In [22]:
def make_dfy(df):
    # 최종 df_y set (product_id onehotencoding)
    n = imputer.fit_transform(df)
    e = encoder.fit_transform(n)
    y_set = pd.DataFrame(e, columns=encoder.get_feature_names(['Y']))
    
    return y_set

In [23]:
def make_df(x, y):
    # 최종 df set
    dfx   = x
    dfy   = make_dfy(y)
    df = pd.merge(dfx, dfy, how='left', left_index=True, right_index=True)
    
    return df

In [42]:
def make_dfx(df):
    # 최종 df_x set(최종 predict를 위한 X_TestF)
    # 제외 컬럼 
    col_except = ['order_id','cstno','week_day','order_hour','is_reordered']
    TestF   = df[list(df.columns.difference(col_except))]
    # X_Test, sample data 와 동일한 순서로 변경
    TestF = TestF[['order_number', 'order_interval', 'add_order'   , 'orgnic_cnt'  ,'orgnic_yn'      ,
                   'bakery'      , 'beverages'     , 'canned goods', 'dairy eggs'  ,'deli', 'frozen' ,
                   'meat seafood', 'pantry'        , 'produce'     , 'avg_interval','total_unq_items',
                   'avg_items'   , 'weekday_yn'    , 'weekend_yn'  , 'morning_yn'  ,'afternoon_yn'   ,
                   'breakfast_yn', 'lunch_yn'      , 'dinner_yn'   ,'order_nums'   ,'week_day_0'     ,
                   'week_day_1'  , 'week_day_2'    , 'week_day_3'  ,'week_day_4'   , 'week_day_5'    ,
                   'week_day_6'  , 'is_reordered_0','is_reordered_1']]

    return TestF

In [43]:
train_df= make_df(train_df2,df_train_y)
test_df = make_df(test_df2,df_test_y)
X_TestF = make_dfx(test_fnl2)

In [38]:
display(train_df)
display(test_df)
display(X_TestF)

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,orgnic_cnt,orgnic_yn,...,Y_5785,Y_5876,Y_7781,Y_8174,Y_8277,Y_8424,Y_8518,Y_9076,Y_9387,Y_9839
0,806422,41116,47,5,18,3.0,4,1,21.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2146033,127113,45,1,10,7.0,24,1,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3055498,140168,45,5,10,26.0,11,1,98.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3267527,60604,52,0,16,8.0,2,1,15.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3125992,136554,46,4,9,5.0,2,0,104.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24419,3184000,174046,100,5,7,2.0,12,1,380.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24420,1754722,6710,100,5,11,3.0,21,1,329.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24421,1076973,150840,100,0,13,0.0,18,1,393.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24422,1247049,99707,100,1,13,3.0,6,1,15.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,orgnic_cnt,...,Y_5785,Y_5876,Y_7781,Y_8174,Y_8277,Y_8424,Y_8518,Y_9076,Y_9387,Y_9839
0,117004,186630,37,4,11,3.0,12,1,21709,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128658,186630,20,2,10,6.0,4,1,21709,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,165882,186630,56,3,15,5.0,10,1,21709,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,180165,186630,17,1,16,4.0,4,1,21709,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,394948,186630,46,5,11,7.0,7,1,21709,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541917,2672845,119134,39,1,14,17.0,4,1,24852,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
541918,2731678,119134,38,5,14,3.0,4,1,24852,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
541919,2860406,119134,9,5,9,8.0,1,1,24852,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
541920,3252013,119134,4,5,10,7.0,2,1,24852,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
select_cols = ['order_number','order_interval','add_order'   ,'orgnic_cnt'  ,'orgnic_yn'      ,
               'bakery'      ,'beverages'     ,'canned goods','dairy eggs'  ,'deli','frozen'  ,
               'meat seafood','pantry'        ,'produce'     ,'avg_interval','total_unq_items',
               'avg_items'   ,'weekday_yn'    ,'weekend_yn'  ,'morning_yn'  ,'afternoon_yn'   ,
               'breakfast_yn','lunch_yn'      ,'dinner_yn'   ,'order_nums'  ,'week_day_0'     ,
               'week_day_1'  ,'week_day_2'    ,'week_day_3'  ,'week_day_4'  ,'week_day_5'     ,
               'week_day_6'  ,'is_reordered_0','is_reordered_1']

for num in list(df_info['product_id'].drop_duplicates()):
    X_Train = train_df[select_cols]
    Y_Train = train_df[f"Y_{num}"]
    X_Test  = test_df[select_cols]
    Y_Test  = test_df[f"Y_{num}"]
    
    # Y_Train sampling
    join_index     = Y_Train[Y_Train  > 0].index.tolist()
    not_join_index = Y_Train[Y_Train == 0].index.tolist()
    
    # 가입 안한 사람들만 sample_ratio만큼 random_sampling(5%)
    random.seed(1234)
    sample_index  = random.sample(not_join_index, round(len(not_join_index)*0.05))
    sample_x_data = X_Train.loc[join_index + sample_index].reset_index(drop = True).fillna(0)
    sample_y_data = Y_Train.loc[join_index + sample_index].reset_index(drop = True).fillna(0)
    
    # sample data pickle 저장
    with open(f'./pickles/sample_x_data{num}.p','wb') as file:
        pickle.dump(sample_x_data ,file)
    with open(f'./pickles/sample_y_data{num}.p','wb') as file:
        pickle.dump(sample_y_data ,file)    
    
    with open('./pickles/X_Test.p','wb') as file:
        pickle.dump(X_Test ,file)
    with open(f'./pickles/Y_Test{num}.p','wb') as file:
        pickle.dump(Y_Test ,file)    

In [49]:
display(sample_x_data)
display(sample_y_data)
display(X_Test)
display(Y_Test)

,order_number,order_interval,add_order,orgnic_cnt,orgnic_yn,bakery,beverages,canned goods,dairy eggs,deli,...,order_nums,week_day_0,week_day_1,week_day_2,week_day_3,week_day_4,week_day_5,week_day_6,is_reordered_0,is_reordered_1
0,43,21.0,4,161.0,1.0,0.0,0.0,22.0,5.0,0.0,...,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,43,5.0,7,96.0,1.0,0.0,0.0,0.0,39.0,15.0,...,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,43,15.0,9,240.0,1.0,0.0,7.0,2.0,56.0,1.0,...,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,46,15.0,8,42.0,1.0,0.0,0.0,0.0,2.0,1.0,...,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,43,7.0,3,134.0,1.0,0.0,0.0,0.0,10.0,0.0,...,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2489,79,3.0,8,48.0,1.0,0.0,1.0,0.0,0.0,0.0,...,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2490,69,3.0,10,139.0,1.0,43.0,41.0,1.0,31.0,2.0,...,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2491,100,7.0,14,251.0,1.0,0.0,6.0,2.0,55.0,0.0,...,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2492,99,6.0,4,312.0,1.0,4.0,0.0,1.0,4.0,0.0,...,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
2489    0.0
2490    0.0
2491    0.0
2492    0.0
2493    0.0
Name: Y_24852, Length: 2494, dtype: float64

,order_number,order_interval,add_order,orgnic_cnt,orgnic_yn,bakery,beverages,canned goods,dairy eggs,deli,...,order_nums,week_day_0,week_day_1,week_day_2,week_day_3,week_day_4,week_day_5,week_day_6,is_reordered_0,is_reordered_1
0,37,3.0,12,18.0,1.0,0.0,67.0,0.0,3.0,0.0,...,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,20,6.0,4,18.0,1.0,0.0,67.0,0.0,3.0,0.0,...,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,56,5.0,10,18.0,1.0,0.0,67.0,0.0,3.0,0.0,...,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,17,4.0,4,18.0,1.0,0.0,67.0,0.0,3.0,0.0,...,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,46,7.0,7,18.0,1.0,0.0,67.0,0.0,3.0,0.0,...,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541917,39,17.0,4,32.0,1.0,0.0,4.0,0.0,7.0,2.0,...,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
541918,38,3.0,4,32.0,1.0,0.0,4.0,0.0,7.0,2.0,...,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
541919,9,8.0,1,32.0,1.0,0.0,4.0,0.0,7.0,2.0,...,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
541920,4,7.0,2,32.0,1.0,0.0,4.0,0.0,7.0,2.0,...,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
541917    1.0
541918    1.0
541919    1.0
541920    1.0
541921    1.0
Name: Y_24852, Length: 541922, dtype: float64

# 6. Model fit(LGBM)
 - LightBGM Classifier import
 - params simulation(pfm check)

In [5]:
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import numpy as np
from numba import jit

In [6]:
class params:
    def __init__(self):
        self.rates = (0.01,0.05)
        self.depths = (5,7)
        self.leaves = (30,50)
        self.params = {}
        self.index  = []

    def get_next(self):
        self._set_next()
        if self.index != []:
            self.params["learning_rate"] = self.rates[self.index[0]]
            self.params["max_depth"]     = self.depths[self.index[1]]
            self.params["num_leaves"]    = self.leaves[self.index[2]]
        else:
            self.params = {}
        return self.params

    def _set_next(self):
        if self.index == []:
            self.index = [0,0,0]
        else:
            self.index[2] = self.index[2] + 1
            if self.index[2] > len(self.leaves)-1:
                self.index[2] = 0            
                self.index[1] = self.index[1] + 1
                if self.index[1] > len(self.depths)-1:
                    self.index[1] = 0
                    self.index[0] = self.index[0] + 1
                    if self.index[0] > len(self.rates)-1:
                        self.index = []

def model_fit(x,y,param):
    model = LGBMClassifier(
                n_estimators = 500,
                learning_rate=param['learning_rate'],
                max_depth=param['max_depth'],
                num_leaves=param['num_leaves'],
                random_state = 1234
            )
    model.fit(x,y)
    return model

def model_pred(x_test,model):
    pred = model.predict(x_test)
    return pred

def _pfm(y_real,y_pred):
    pfm = f1_score(y_real,y_pred)
    return pfm

In [7]:
%%time
for num in list(df_info['product_id'].drop_duplicates()):
    tstamp(f"\r{num} "," ~ ")
    # 데이터 불러오기
    with open(f"./pickles/sample_x_data{num}.p",'rb') as file:
        globals()[f"sx_{num}"] = pickle.load(file)
    with open(f"./pickles/sample_y_data{num}.p",'rb') as file:
        globals()[f"sy_{num}"] = pickle.load(file)
    with open("./pickles/X_Test.p",'rb') as file:
        X_Test = pickle.load(file)
    with open(f"./pickles/Y_Test{num}.p",'rb') as file:
        globals()[f"Y_Test{num}"] = pickle.load(file)    
    
    best_model = {}
    p = params()
    while(True):
        param  = p.get_next()
        if param == {}:
            break
        else:
            model  = model_fit(globals()[f"sx_{num}"],globals()[f"sy_{num}"],param)
            pred   = model_pred(X_Test,model)
            pfm    = _pfm(globals()[f"Y_Test{num}"],pred)
            if pfm > best_model.get("pfm",0):
                best_model["model"]  = model
                best_model["params"] = params
                best_model["pfm"]    = pfm
                
    # Product id별 최적모델 pickle로 내보내기            
    with open(f'./md_lgbm/lgbm{num}.p','wb') as file:
        pickle.dump(best_model ,file)
    tstamp(None,"\n")   
print("completed")    

21709  [04:32:43 PM]  ~  [04:33:11 PM] 
20995  [04:33:11 PM]  ~  [04:33:40 PM] 
33000  [04:33:40 PM]  ~  [04:34:10 PM] 
7781  [04:34:10 PM]  ~  [04:34:37 PM] 
47144  [04:34:37 PM]  ~  [04:35:06 PM] 
41220  [04:35:06 PM]  ~  [04:35:38 PM] 
41787  [04:35:38 PM]  ~  [04:36:07 PM] 
4210  [04:36:07 PM]  ~  [04:36:38 PM] 
38689  [04:36:38 PM]  ~  [04:37:10 PM] 
42736  [04:37:10 PM]  ~  [04:37:39 PM] 
196  [04:37:39 PM]  ~  [04:38:04 PM] 
16759  [04:38:04 PM]  ~  [04:38:34 PM] 
9387  [04:38:34 PM]  ~  [04:39:26 PM] 
4799  [04:39:26 PM]  ~  [04:39:56 PM] 
34358  [04:39:56 PM]  ~  [04:40:25 PM] 
23909  [04:40:25 PM]  ~  [04:40:56 PM] 
43122  [04:40:56 PM]  ~  [04:41:25 PM] 
3957  [04:41:25 PM]  ~  [04:41:55 PM] 
27156  [04:41:55 PM]  ~  [04:42:22 PM] 
9839  [04:42:22 PM]  ~  [04:42:48 PM] 
46906  [04:42:48 PM]  ~  [04:43:15 PM] 
15290  [04:43:15 PM]  ~  [04:43:43 PM] 
28199  [04:43:43 PM]  ~  [04:44:16 PM] 
18465  [04:44:16 PM]  ~  [04:44:43 PM] 
29487  [04:44:43 PM]  ~  [04:45:14 PM] 
11520  [

# 7. 최종 test_y Predict
 - X_TestF을 Model predict 하여 Score 값이 가장 큰 product_id 채택!
 - test_y.csv 내보내기

In [44]:
display(X_TestF)

,order_number,order_interval,add_order,orgnic_cnt,orgnic_yn,bakery,beverages,canned goods,dairy eggs,deli,...,order_nums,week_day_0,week_day_1,week_day_2,week_day_3,week_day_4,week_day_5,week_day_6,is_reordered_0,is_reordered_1
0,38,13.0,9,69.0,1.0,0.0,18.0,0.0,33.0,1.0,...,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,38,7.0,4,147.0,1.0,0.0,60.0,0.0,45.0,0.0,...,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,45,30.0,5,8.0,1.0,0.0,20.0,0.0,48.0,0.0,...,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,39,13.0,15,106.0,1.0,0.0,17.0,1.0,30.0,0.0,...,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,42,17.0,1,89.0,1.0,0.0,5.0,2.0,7.0,0.0,...,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16121,46,8.0,2,38.0,1.0,4.0,7.0,0.0,3.0,1.0,...,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
16122,43,0.0,1,5.0,1.0,0.0,0.0,0.0,3.0,0.0,...,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
16123,49,3.0,2,24.0,1.0,13.0,0.0,0.0,29.0,0.0,...,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
16124,43,10.0,1,97.0,1.0,0.0,0.0,0.0,14.0,0.0,...,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [110]:
%%time
for num in list(df_info['product_id'].drop_duplicates()):
    tstamp(f"\r{num} "," ~ ")
    # model 불러와서 score 생성
    globals()[f"md{num}"] = pickle.load(open(f"./md_lgbm/lgbm{num}.p",'rb'))['model']
    pred = pd.DataFrame(globals()[f"md{num}"].predict_proba(X_TestF))
    pred.columns = ["pred_0",f"{num}"]
    pred.drop(columns = ['pred_0'], inplace=True)
                
    # productid별 Score값 pickle로 내보내기            
    with open(f'./score/score{num}.p','wb') as file:
        pickle.dump(pred ,file)
    tstamp(None,"\n")   
print("completed")    

21709  [06:45:15 PM]  ~  [06:45:15 PM] 
20995  [06:45:15 PM]  ~  [06:45:15 PM] 
33000  [06:45:15 PM]  ~  [06:45:15 PM] 
7781  [06:45:15 PM]  ~  [06:45:15 PM] 
47144  [06:45:15 PM]  ~  [06:45:15 PM] 
41220  [06:45:15 PM]  ~  [06:45:15 PM] 
41787  [06:45:15 PM]  ~  [06:45:15 PM] 
4210  [06:45:15 PM]  ~  [06:45:15 PM] 
38689  [06:45:15 PM]  ~  [06:45:15 PM] 
42736  [06:45:15 PM]  ~  [06:45:15 PM] 
196  [06:45:15 PM]  ~  [06:45:16 PM] 
16759  [06:45:16 PM]  ~  [06:45:16 PM] 
9387  [06:45:16 PM]  ~  [06:45:16 PM] 
4799  [06:45:16 PM]  ~  [06:45:16 PM] 
34358  [06:45:16 PM]  ~  [06:45:16 PM] 
23909  [06:45:16 PM]  ~  [06:45:16 PM] 
43122  [06:45:16 PM]  ~  [06:45:16 PM] 
3957  [06:45:16 PM]  ~  [06:45:16 PM] 
27156  [06:45:16 PM]  ~  [06:45:16 PM] 
9839  [06:45:16 PM]  ~  [06:45:16 PM] 
46906  [06:45:16 PM]  ~  [06:45:16 PM] 
15290  [06:45:16 PM]  ~  [06:45:16 PM] 
28199  [06:45:16 PM]  ~  [06:45:16 PM] 
18465  [06:45:16 PM]  ~  [06:45:16 PM] 
29487  [06:45:16 PM]  ~  [06:45:16 PM] 
11520  [

In [115]:
%%time
# prodict_id별 Score concat
S통합 = pd.concat([pd.read_pickle(f"./score/score{num}.p") for num in list(df_info['product_id'].drop_duplicates())], axis=1)

# max score find
max_idx = S통합.idxmax(axis=1)
S통합['max_Score']  = max_idx
S통합['product_id'] = S통합['max_Score']

CPU times: user 84.5 ms, sys: 15.5 ms, total: 100 ms
Wall time: 99.9 ms


In [116]:
S통합

,21709,20995,33000,7781,47144,41220,41787,4210,38689,42736,...,16797,47626,47766,47209,21903,21137,13176,24852,max_Score,product_id
0,0.120085,0.003697,0.000191,0.005316,0.260513,0.000248,0.000225,0.016439,0.330891,0.000378,...,0.207254,0.546416,0.222877,0.158180,0.900261,0.609682,0.293892,0.406739,21903,21903
1,0.236713,0.002372,0.000044,0.028047,0.035062,0.000313,0.045015,0.005275,0.068376,0.000491,...,0.059737,0.394166,0.330741,0.419935,0.069869,0.355755,0.643849,0.440008,35221,35221
2,0.658698,0.002247,0.000050,0.001312,0.412388,0.000167,0.020061,0.147957,0.029071,0.003263,...,0.232291,0.136967,0.058971,0.054506,0.084582,0.198640,0.262678,0.503657,23909,23909
3,0.126737,0.004542,0.002090,0.011651,0.005672,0.001449,0.000432,0.025336,0.038875,0.000831,...,0.126525,0.539490,0.311329,0.355799,0.675951,0.568040,0.330245,0.182842,44359,44359
4,0.031298,0.002439,0.000029,0.001270,0.031287,0.000377,0.001644,0.014945,0.073885,0.000684,...,0.232587,0.185933,0.721434,0.311260,0.374245,0.539393,0.669023,0.877094,24852,24852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16121,0.078980,0.050048,0.000015,0.000122,0.066687,0.000886,0.054088,0.009742,0.002103,0.000107,...,0.434666,0.191345,0.322322,0.167242,0.320398,0.254527,0.303024,0.837577,5077,5077
16122,0.001254,0.002368,0.000007,0.000009,0.083635,0.000186,0.872439,0.308496,0.002283,0.000032,...,0.469560,0.162049,0.075533,0.054632,0.009623,0.327568,0.202601,0.868057,41787,41787
16123,0.001452,0.007850,0.021854,0.000087,0.578516,0.000059,0.134650,0.603878,0.016276,0.001249,...,0.243126,0.243132,0.025603,0.098161,0.003606,0.296713,0.282036,0.790048,5077,5077
16124,0.006042,0.023171,0.000228,0.000140,0.014726,0.003312,0.000301,0.006365,0.020260,0.000428,...,0.458049,0.222691,0.730633,0.141491,0.074878,0.489379,0.680201,0.933465,24852,24852


In [133]:
test_y = pd.DataFrame(S통합['product_id'])

In [137]:
test_y

,product_id
0,21903
1,35221
2,23909
3,44359
4,24852
...,...
16121,5077
16122,41787
16123,5077
16124,24852


In [138]:
test_y.to_csv('./test_y.csv',index=False)